In [12]:
import re
import numpy as np
import pandas as pd
from rapidfuzz import fuzz, process
import re
from sentence_transformers import SentenceTransformer, util

In [9]:
def _normalize_series(s: pd.Series, extra_suffixes=None) -> pd.Series:
    if extra_suffixes is None:
        extra_suffixes = []
    suffixes = [
        "inc", "llc", "ltd", "corp", "co", "company", "corporation", "limited",
        "incorporated", "plc", "gmbh", "sa", "s.a.", "ag", "pte", "bv", "srl",
        "sro", "oy", "aps", "as", "ab", "kk", "k.k.", "sl", "sas", "spa"
    ] + [s.lower() for s in extra_suffixes]
    suffix_re = re.compile(r"\b(" + "|".join(map(re.escape, suffixes)) + r")\b")

    def _norm(x):
        if pd.isna(x):
            return ""
        x = str(x).lower()
        x = re.sub(r"[^a-z0-9\s]", " ", x)
        x = suffix_re.sub(" ", x)
        x = re.sub(r"\bthe\b", " ", x)
        x = re.sub(r"\s+", " ", x).strip()
        return x

    return s.apply(_norm)

In [10]:
def fuzzy_match_company_lists(
    df_left: pd.DataFrame,
    df_right: pd.DataFrame,
    left_name_col: str = "CompanyName",
    right_name_col: str = "CompanyName",
    left_id_col: str = "CompanyID",
    right_id_col: str  = "CompanyID",
    n_top: int = 3,
    threshold: int = 80,
    output_mode: str = "concat",  # "concat" or "rows"
    scorer=fuzz.token_sort_ratio,
    extra_suffixes: list[str]  = None
) -> pd.DataFrame:
    if left_name_col not in df_left.columns or right_name_col not in df_right.columns:
        raise ValueError("Name columns not found in one or both DataFrames.")

    if left_id_col is None or left_id_col not in df_left.columns:
        left_id_col = None
    if right_id_col is None or right_id_col not in df_right.columns:
        right_id_col = None

    left = df_left.copy()
    right = df_right.copy()
    left["_norm"]  = _normalize_series(left[left_name_col],  extra_suffixes)
    right["_norm"] = _normalize_series(right[right_name_col], extra_suffixes)

    left_names_norm  = left["_norm"].tolist()
    right_names_norm = right["_norm"].tolist()

    # --- Call cdist without 'limit' to be compatible with older versions
    results_raw = process.cdist(
        left_names_norm,
        right_names_norm,
        scorer=scorer,
        score_cutoff=threshold
    )

     # --- Normalize into list[list[(j, score)]]
    matches_per_left: list[list[tuple[int, float]]] = []
    if isinstance(results_raw, np.ndarray):
        for row in results_raw:
            if n_top <= 0:
                matches_per_left.append([])
                continue
            cand_idx = np.flatnonzero(row >= threshold)
            if cand_idx.size == 0:
                matches_per_left.append([])
                continue
            top_idx = cand_idx[np.argsort(row[cand_idx])[-n_top:]][::-1]
            matches_per_left.append([(int(j), float(row[j])) for j in top_idx])
    else:
        for match_list in results_raw:
            match_list = sorted(match_list, key=lambda t: t[1], reverse=True)[:n_top]
            matches_per_left.append([(int(j), float(score)) for (j, score, _choice) in match_list])

    if output_mode == "concat":
        matches_col = []
        for i, match_list in enumerate(matches_per_left):
            items = []
            left_id_val = (left.iloc[i][left_id_col] if left_id_col else pd.NA)
            for j, score in match_list:
                right_id_val = (right.iloc[j][right_id_col] if right_id_col else pd.NA)
                # skip self-match by ID
                if left_id_col and right_id_col and pd.notna(left_id_val) and left_id_val == right_id_val:
                    continue
                right_name = right.iloc[j][right_name_col]
                items.append(f"{right_name} ({int(score)})")
            matches_col.append("; ".join(items))
        out = left.drop(columns=["_norm"])
        out["Matches"] = matches_col
        return out

    elif output_mode == "rows":
        rows = []
        for i, match_list in enumerate(matches_per_left):
            for j, score in match_list:
                left_id_val  = (left.iloc[i][left_id_col]  if left_id_col  else pd.NA)
                right_id_val = (right.iloc[j][right_id_col] if right_id_col else pd.NA)

                # Skip same ID matches
                if left_id_col and right_id_col and pd.notna(left_id_val) and left_id_val == right_id_val:
                    continue

                rows.append({
                    "LeftCompanyID":  left_id_val,
                    "LeftName":       left.iloc[i][left_name_col],
                    "RightCompanyID": right_id_val,
                    "RightName":      right.iloc[j][right_name_col],
                    "MatchScore":     int(score)
                })

        cols = ["LeftCompanyID", "LeftName", "RightCompanyID", "RightName", "MatchScore"]
        return pd.DataFrame(rows, columns=cols)

    else:
        raise ValueError("output_mode must be 'concat' or 'rows'.")

In [57]:
df_out.to_csv("fuzzy_matches.csv", index=False, encoding="utf-8-sig")

In [13]:
def semantic_match_company_lists(
    df_left: pd.DataFrame,
    df_right: pd.DataFrame,
    left_name_col: str = "CompanyName",
    right_name_col: str = "CompanyName",
    left_id_col: str = "CompanyID",
    right_id_col: str = "CompanyID",
    n_top: int = 3,
    threshold: float = 0.75,     # cosine similarity cutoff (0–1)
    output_mode: str = "concat", # or "rows"
    model_name: str = "sentence-transformers/all-MiniLM-L6-v2",
    extra_suffixes: list[str] = None
) -> pd.DataFrame:
    """
    Compare company names semantically using a SentenceTransformer model.
    """
    # --- Load model
    model = SentenceTransformer(model_name)

    # --- Normalize names
    left = df_left.copy()
    right = df_right.copy()
    left["_norm"]  = _normalize_series(left[left_name_col],  extra_suffixes)
    right["_norm"] = _normalize_series(right[right_name_col], extra_suffixes)

    # --- Encode all company names
    left_embs  = model.encode(left["_norm"].tolist(), convert_to_tensor=True, show_progress_bar=True)
    right_embs = model.encode(right["_norm"].tolist(), convert_to_tensor=True, show_progress_bar=True)

    # --- Compute cosine similarity matrix
    cosine_scores = util.cos_sim(left_embs, right_embs).cpu().numpy()  # shape [n_left, n_right]

    # --- Build output
    if output_mode == "concat":
        matches_col = []
        for i in range(cosine_scores.shape[0]):
            row_scores = cosine_scores[i]
            top_idx = np.argsort(row_scores)[::-1]  # descending
            best = []
            for j in top_idx[:n_top]:
                score = row_scores[j]
                if score < threshold:
                    continue
                # skip self-match if IDs identical
                left_id  = left.at[i, left_id_col]  if left_id_col in left else None
                right_id = right.at[j, right_id_col] if right_id_col in right else None
                if left_id_col and right_id_col and left_id == right_id:
                    continue
                best.append(f"{right.at[j, right_name_col]} ({score:.3f})")
            matches_col.append("; ".join(best))
        out = left.drop(columns=["_norm"])
        out["Matches"] = matches_col
        return out

    elif output_mode == "rows":
        rows = []
        for i in range(cosine_scores.shape[0]):
            row_scores = cosine_scores[i]
            top_idx = np.argsort(row_scores)[::-1][:n_top]
            for j in top_idx:
                score = row_scores[j]
                if score < threshold:
                    continue
                left_id  = left.at[i, left_id_col]  if left_id_col in left else None
                right_id = right.at[j, right_id_col] if right_id_col in right else None
                if left_id_col and right_id_col and left_id == right_id:
                    continue
                rows.append({
                    "LeftCompanyID":  left_id,
                    "LeftName":       left.at[i, left_name_col],
                    "RightCompanyID": right_id,
                    "RightName":      right.at[j, right_name_col],
                    "SemanticScore":  round(float(score), 4)
                })
        return pd.DataFrame(rows, columns=["LeftCompanyID","LeftName","RightCompanyID","RightName","SemanticScore"])
    else:
        raise ValueError("output_mode must be 'concat' or 'rows'.")

In [39]:
# df1 = pd.DataFrame([{"CompanyID":1, "CompanyName":"Wealth Abound."}])
df2 = pd.read_excel("companies.xlsx",sheet_name=0)
df1 = pd.read_excel("companies.xlsx",sheet_name=0)


df_result = semantic_match_company_lists(
    df_left=df1,
    df_right=df2,
    left_name_col="CompanyName",
    right_name_col="CompanyName",
    left_id_col="CompanyID",
    right_id_col="CompanyID",
    n_top=3,               # how many matches to show per company
    threshold=0.85,         # cosine similarity cutoff (0–1)
    output_mode="concat"     # or "concat"
)

c:\users\cmurp\appdata\local\programs\python\python39\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Batches:   0%|          | 0/47 [00:00<?, ?it/s]

Batches:   0%|          | 0/47 [00:00<?, ?it/s]

In [40]:
df_result

,CompanyID,CompanyName,Matches
0,1,&Partners,
1,2,103 Advisory Group LLC,
2,3,180 Wealth Advisors LLC,
3,4,1858 Wealth Management LLC,
4,5,1919 Investment Counsel,
...,...,...,...
1476,1477,Yahoo!,
1477,1478,Yardley Wealth Management LLC,
1478,1479,Yarra Capital Management (P),
1479,1480,YoungsAG,


In [12]:
df_result.to_csv("semantic_matches.csv", index=False, encoding="utf-8-sig")

In [14]:
#### COMBINE DFs
import pandas as pd

def join_match_results(
    df_fuzzy: pd.DataFrame,
    df_semantic: pd.DataFrame,
    id_col: str = "LeftCompanyID"
) -> pd.DataFrame:
    """
    Joins fuzzy and semantic match DataFrames that share the same format (both 'concat' or both 'rows').

    Auto-detects format based on number of columns:
        - concat format -> fewer columns (usually <= 4)
        - rows format   -> more columns (usually > 4)
    """

    # --- Detect format automatically ---
    format_type = "rows" if df_fuzzy.shape[1] > 4 else "concat"

    # --- Join for CONCAT format ---
    if format_type == "concat":
        merged = pd.merge(
            df_fuzzy[[id_col,"CompanyName", "Matches"]],
            df_semantic[[id_col, "Matches"]],
            on=id_col,
            how="outer",
            suffixes=("_fuzzy", "_semantic")
        )
        return merged

    # --- Join for ROWS format ---
    else:  # rows mode
        merged = pd.merge(
            df_fuzzy,
            df_semantic,
            on=id_col,
            how="outer",
            suffixes=("_fuzzy", "_semantic")
        )

        # Optional: reorder for readability if expected columns exist
        cols = [
            id_col,
            "LeftName_fuzzy",
            "RightName_fuzzy",
            "MatchScore",
            "RightName_semantic",
            "SemanticScore"
        ]
        return merged[[c for c in cols if c in merged.columns]]


In [24]:

df1 = pd.read_excel("companies.xlsx",sheet_name=0)
df2 = pd.read_excel("companies.xlsx",sheet_name=0)
# df2 = pd.DataFrame([{"CompanyID":1, "CompanyName":"Beacon Advisor"}])

# df2 = pd.read_excel("companies.xlsx",sheet_name=0)
# df1 = pd.DataFrame([{"CompanyID":1, "CompanyName":"Beacon Advisor"}])


df_fuzzy = fuzzy_match_company_lists(
    df_left=df1, 
    df_right=df2,
    left_name_col="CompanyName", left_id_col="CompanyID",
    right_name_col="CompanyName", right_id_col="CompanyID",
    n_top=3, threshold=85, 
    output_mode="concat"  #"concat" or "rows"
)

In [16]:
df_semantic = semantic_match_company_lists(
    df_left=df1,
    df_right=df2,
    left_name_col="CompanyName",
    right_name_col="CompanyName",
    left_id_col="CompanyID",
    right_id_col="CompanyID",
    n_top=3,               # how many matches to show per company
    threshold=0.85,         # cosine similarity cutoff (0–1)
    output_mode="concat"     # or "concat"
    model_name="sentence-transformers/all-MiniLM-L6-v2",
)

c:\users\cmurp\appdata\local\programs\python\python39\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Batches:   0%|          | 0/47 [00:00<?, ?it/s]

Batches:   0%|          | 0/47 [00:00<?, ?it/s]

In [87]:
combined = join_match_results(df_fuzzy, df_semantic, id_col='CompanyID')

In [76]:
combined[combined['CompanyID'] == 17]

,CompanyID,CompanyName,Matches_fuzzy,Matches_semantic
16,17,Abound Wealth,,


In [88]:
combined.to_csv("combined_concat3.csv", index=False, encoding="utf-8-sig")

In [25]:


df_finetune = semantic_match_company_lists(
    df_left=df1,
    df_right=df2,
    left_name_col="CompanyName",
    right_name_col="CompanyName",
    left_id_col="CompanyID",
    right_id_col="CompanyID",
    n_top=3,
    threshold=0.85,
    output_mode="concat",
    model_name="finetuned-company-matcher",
)

Batches:   0%|          | 0/47 [00:00<?, ?it/s]

Batches:   0%|          | 0/47 [00:00<?, ?it/s]

In [20]:
df_finetune

,LeftCompanyID,LeftName,RightCompanyID,RightName,SemanticScore
0,9,4Thought Financial Group,569,FourThought Financial LLC,0.9644
1,18,Abrams Bison Investments L.L.C.,19,Abrams Bison Investments LLC,0.9576
2,19,Abrams Bison Investments LLC,18,Abrams Bison Investments L.L.C.,0.9576
3,25,Accredited Investors,26,Accredited Wealth Management LLC,0.7426
4,26,Accredited Wealth Management LLC,25,Accredited Investors,0.7426
...,...,...,...,...,...
587,1459,William Blair Investment Management LLC,1458,William Blair & Company LLC,0.8455
588,1459,William Blair Investment Management LLC,1460,William Blair LLC,0.8455
589,1460,William Blair LLC,1458,William Blair & Company LLC,1.0000
590,1460,William Blair LLC,1459,William Blair Investment Management LLC,0.8455


In [26]:
combined = join_match_results(df_fuzzy, df_finetune, id_col='CompanyID')

In [27]:
combined.to_csv("combined_concat3.csv", index=False, encoding="utf-8-sig")